# Build a Question/Answering system over SQL data #

https://python.langchain.com/v0.2/docs/tutorials/sql_qa/

In [1]:
%%capture --no-stderr
%pip install --upgrade --quiet  langchain langchain-community langchain-openai

In [2]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass()

# Comment out the below to opt-out of using LangSmith in this notebook. Not required.
if not os.environ.get("LANGCHAIN_API_KEY"):
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
    os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [38]:

from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

## Chains ##

### Convert question to SQL query ###

In [5]:
%pip install -qU langchain-openai

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")



Note: you may need to restart the kernel to use updated packages.


In [6]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many employees are there"})
response

'SELECT COUNT("EmployeeId") AS "TotalEmployees" FROM Employee'

In [7]:
db.run(response)

'[(8,)]'

In [8]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

In [11]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
chain = write_query | execute_query
chain.invoke({"question": "How many employees are there"})

'[(8,)]'

In [12]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer_prompt
    | llm
    | StrOutputParser()
)

chain.invoke({"question": "How many employees are there"})

'There are 8 employees.'

In [15]:
chain.invoke({"question": "List all the employees grouped by their city. Use bulletpoints to show the result"})

'- Calgary\n  - Nancy Edwards\n  - Jane Peacock\n  - Margaret Park\n  - Steve Johnson\n  - Michael Mitchell\n- Edmonton\n  - Andrew Adams\n- Lethbridge\n  - Robert King\n  - Laura Callahan'

## AGENTS ##

https://python.langchain.com/v0.2/docs/tutorials/sql_qa/#agents

In [16]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

tools

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x10600f890>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x10600f890>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x10600f890>),
 QuerySQLCheckerTool(description='Use this tool to double check if your 

In [17]:
from langchain_core.messages import SystemMessage

SQL_PREFIX = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables."""

system_message = SystemMessage(content=SQL_PREFIX)

In [18]:
%%capture --no-stderr
%pip install --upgrade --quiet langgraph

In [19]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools, messages_modifier=system_message)

In [20]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content="Which country's customers spent the most?")]}
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_I1N5clhNgNuNRGZl2dZToO3L', 'function': {'arguments': '{"query":"SELECT c.Country, SUM(i.Total) as Total_spent FROM customers c JOIN invoices i ON c.CustomerId = i.CustomerId GROUP BY c.Country ORDER BY Total_spent DESC LIMIT 1"}', 'name': 'sql_db_query'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 557, 'total_tokens': 610}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6acf49d2-6386-4a38-a78b-7cd301030387-0', tool_calls=[{'name': 'sql_db_query', 'args': {'query': 'SELECT c.Country, SUM(i.Total) as Total_spent FROM customers c JOIN invoices i ON c.CustomerId = i.CustomerId GROUP BY c.Country ORDER BY Total_spent DESC LIMIT 1'}, 'id': 'call_I1N5clhNgNuNRGZl2dZToO3L'}], usage_metadata={'input_tokens': 557, 'output_tokens': 53, 'total_tokens': 610})]}}
----
{'tools': 

In [21]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content="Describe the playlisttrack table")]}
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1WBuqCVOl0Ked2jZRVIDIJva', 'function': {'arguments': '{"table_names":"playlisttrack"}', 'name': 'sql_db_schema'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 554, 'total_tokens': 571}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-aa274f52-d474-45f6-8883-4f06d0ccc8f4-0', tool_calls=[{'name': 'sql_db_schema', 'args': {'table_names': 'playlisttrack'}, 'id': 'call_1WBuqCVOl0Ked2jZRVIDIJva'}], usage_metadata={'input_tokens': 554, 'output_tokens': 17, 'total_tokens': 571})]}}
----
{'tools': {'messages': [ToolMessage(content="Error: table_names {'playlisttrack'} not found in database", name='sql_db_schema', tool_call_id='call_1WBuqCVOl0Ked2jZRVIDIJva')]}}
----
{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_rJf58jfZB0M9o0RXM3l

## Dealing with high-cardinality columns ##

https://python.langchain.com/v0.2/docs/tutorials/sql_qa/#dealing-with-high-cardinality-columns

In [23]:
import ast
import re


def query_as_list(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
    return list(set(res))


artists = query_as_list(db, "SELECT Name FROM Artist")
albums = query_as_list(db, "SELECT Title FROM Album")
print(artists)
print(albums)
albums[:5]

['Stevie Ray Vaughan & Double Trouble', 'Bruce Dickinson', 'A Cor Do Som', 'Gerald Moore', 'Battlestar Galactica', 'Raimundos', 'Foo Fighters', 'Elis Regina', 'Christina Aguilera featuring BigElf', 'Orchestre Révolutionnaire et Romantique & John Eliot Gardiner', "Christopher O'Riley", 'Sabotage E Instituto', 'Hilary Hahn, Jeffrey Kahane, Los Angeles Chamber Orchestra & Margaret Batjer', 'Vinícius De Moraes & Baden Powell', 'Cidade Negra', 'Jota Quest', 'Funk Como Le Gusta', 'Hermeto Pascoal', 'Black Label Society', 'Ben Harper', 'Os Paralamas Do Sucesso', 'Xis', 'Temple of the Dog', 'The Tea Party', 'The Clash', 'Chris Cornell', 'Julian Bream', 'Motörhead & Girlschool', "Jack's Mannequin & Mick Fleetwood", 'The Posies', 'Cake', 'The  Cellists of The Berlin Philharmonic', 'Milton Nascimento', "Paul D'Ianno", 'Red Hot Chili Peppers', 'C. Monteverdi, Nigel Rogers - Chiaroscuro; London Baroque; London Cornett & Sackbu', 'Wilhelm Kempff', 'Yo-Yo Ma', 'Calexico', 'Chico Buarque', 'Chicago Sy

['Living After Midnight',
 "Great Recordings of the Century: Paganini's  Caprices",
 'Alcohol Fueled Brewtality Live! [Disc ]',
 'SCRIABIN: Vers la flamme',
 'Album Of The Year']

In [25]:
%pip install faiss-cpu

  Using cached faiss_cpu-1.8.0-cp312-cp312-macosx_10_14_x86_64.whl.metadata (3.6 kB)
Using cached faiss_cpu-1.8.0-cp312-cp312-macosx_10_14_x86_64.whl (7.4 MB)
Note: you may need to restart the kernel to use updated packages.


In [26]:
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

vector_db = FAISS.from_texts(artists + albums, OpenAIEmbeddings())
retriever = vector_db.as_retriever(search_kwargs={"k": 5})
description = """Use to look up values to filter on. Input is an approximate spelling of the proper noun, output is \
valid proper nouns. Use the noun most similar to the search."""
retriever_tool = create_retriever_tool(
    retriever,
    name="search_proper_nouns",
    description=description,
)

In [32]:
print(retriever_tool.invoke("AC/CD"))

AC/DC

A-Sides

Live [Disc ]

Garage Inc. (Disc )

Black Album


In [33]:
system = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

You have access to the following tables: {table_names}

If you need to filter on a proper noun, you must ALWAYS first look up the filter value using the "search_proper_nouns" tool!
Do not try to guess at the proper name - use this function to find similar ones.""".format(
    table_names=db.get_usable_table_names()
)

system_message = SystemMessage(content=system)

tools.append(retriever_tool)

agent = create_react_agent(llm, tools, messages_modifier=system_message)

In [36]:
for s in agent.stream(
    {"messages": [HumanMessage(content="Which albums does acdc have?")]}
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_A6M1xgdbUgIAwDrhq5lQmGQu', 'function': {'arguments': '{"query":"acdc"}', 'name': 'search_proper_nouns'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 671, 'total_tokens': 689}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-0fd087a1-cdc8-46fa-a050-978fa7a8562c-0', tool_calls=[{'name': 'search_proper_nouns', 'args': {'query': 'acdc'}, 'id': 'call_A6M1xgdbUgIAwDrhq5lQmGQu'}], usage_metadata={'input_tokens': 671, 'output_tokens': 18, 'total_tokens': 689})]}}
----
{'tools': {'messages': [ToolMessage(content='AC/DC\n\nBlack Sabbath\n\nBlack Sabbath\n\nIron Maiden\n\nIron Maiden', name='search_proper_nouns', tool_call_id='call_A6M1xgdbUgIAwDrhq5lQmGQu')]}}
----
{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_NxBIUXrynHMj0cgQQi9i